Import needed libraries.

- BeautifulSoup is used for web scrapping purposes

- requests, urllib, and ssl are used for making api request

- json used to deal with json format

- pandas used for dealing with data and exporting

Potential Additions: Extracting Keywords and Citations

- However citations just give another article ID.

In [1]:
from bs4 import BeautifulSoup
import requests, urllib, ssl, json
import pandas as pd

API Request Key

In [2]:
api_req_url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=100&sort=relevance&term="
keyword = "KEYWORD"

Ask for user input

In [3]:
keyword = str(input("Enter the keyword required: \n"))
api_req_url = api_req_url + keyword

Enter the keyword required: 
immport


Need to verify your HTTP Certificate and then make request.

In [4]:
_create_unverified_https_context = ssl._create_unverified_context
request = urllib.request.urlopen(api_req_url).read()

Now need to be able to access JSON data and parse specific things

TODO: Could add a feature for inputting how many articles wanted and also wanted/totalmatch ratio

In [5]:
data = json.loads(request)

article_ids = data['esearchresult']['idlist']
article_ids

['29485622',
 '32411773',
 '24791905',
 '31272390',
 '32504750',
 '28057685',
 '30852461',
 '30977847',
 '30304689',
 '32707495',
 '32686362',
 '32675942',
 '32600423',
 '32375704',
 '32355728',
 '32351547',
 '32310824',
 '32068352',
 '32019546',
 '32296685',
 '32214002',
 '31821170',
 '31921619',
 '30832680',
 '30516029',
 '30594555',
 '30398470',
 '27377652',
 '27130330',
 '20839340',
 '26112029',
 '20460173',
 '32283555',
 '28851925',
 '25857935',
 '24862987',
 '24161793',
 '22959960',
 '20044059']

Main logic here.

Basically scrapes the API return and then gets key information including the abstract that can be used later

In [6]:
def get_elements(soup_object):
    
    authors = getAuthors(soup_object)
    title = getTitle(soup_object)
    abstract = getAbstract(soup_object)
    pubmed_id = getID(soup_object)
    
    journal_volume = ""
    journal_title = ""
    journal_date = ""
    
    if soup_object.find("JournalIssue"):
        if soup_object.find("JournalIssue").find("Volume"):
            journal_volume = soup_object.find("JournalIssue").find("Volume").text
    if soup_object.find("Title"):
        journal_title = soup_object.find("Title").text
    if soup_object.find("JournalIssue").find("Year") and soup_object.find("JournalIssue").find("Month") and soup_object.find("JournalIssue").find("Day"):
        journal_date = soup_object.find("JournalIssue").find("Year").text + " " + soup_object.find("JournalIssue").find("Month").text + " " + soup_object.find("JournalIssue").find("Day").text
        
    doi = soup_object.find_all('ELocationID')
    doi_str = ""
    if len(doi)>1:
        print(doi[0])
        if 'doi' in str(doi[0]):
            doi = doi[0].text
            doi_str += "DOI "
            doi_str += doi
            doi_str += "."
        elif 'doi' in str(doi[1]):
            doi = doi[1].text
            doi_str += "DOI "
            doi_str += doi
            doi_str += "."
        elif len(doi) == 1:
            if 'doi' in str(doi[0]):
                doi = doi[0].text
                doi_str += "DOI "
                doi_str += doi
                doi_str += "."
            elif 'pii' in str(doi[0]):
                doi = doi[0].text
                doi_str += "PII "
                doi_str += doi
                doi_str += "."
    print(doi_str)
    
    row_list = []
    row_list.append(authors)
    row_list.append(title)
    row_list.append(journal_title)
    row_list.append(journal_volume)
    row_list.append(journal_date)
    row_list.append(pubmed_id)
    row_list.append(abstract)
    row_list.append(doi_str)
    
    return row_list
    
def getAuthors(soup_object):
    authors = ""
    authorlist = soup_object.find("AuthorList")
    ##print(authorlist)
    
    if authorlist: 
        for i in range(len(authorlist.find_all("LastName"))):
            firstName = authorlist.find_all("ForeName")[i].text
            authors = authors + firstName
            lastName = authorlist.find_all("LastName")[i].text
            authors = authors + " " + lastName
            
            if i != len(authorlist.find_all("LastName")) - 1:
                authors = authors + ", "
    print(authors)
    return authors

def getTitle(soup_object):
    title = ""
    if soup_object.find("AricleTitle"):
        title_element = soup_object.find("ArticleTitle")
        title = title_element.text

    print(title)
    return title
    
def getAbstract(soup_object):
    abstract = ""
    if soup_object.find("AbstractText"):
        abstract = soup_object.find("AbstractText").text
    print(abstract)
    return abstract

def getID(soup_object):
    idInfo = ""
    if soup_object.find("PMID"):
        idInfo = "PMID: "
        idInfo += soup_object.find("PMID").text
    return idInfo

Just a test for one of the articles.

In [7]:
get_url = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id=29485622"

    
_create_unverified_https_context = ssl._create_unverified_context
response = requests.get(get_url)

soup_object = BeautifulSoup(response.content, "xml")
one_article = get_elements(soup_object)

Sanchita Bhattacharya, Patrick Dunn, Cristel G Thomas, Barry Smith, Henry Schaefer, Jieming Chen, Zicheng Hu, Kelly A Zalocusky, Ravi D Shankar, Shai S Shen-Orr, Elizabeth Thomson, Jeffrey Wiser, Atul J Butte

Immunology researchers are beginning to explore the possibilities of reproducibility, reuse and secondary analyses of immunology data. Open-access datasets are being applied in the validation of the methods used in the original studies, leveraging studies for meta-analysis, or generating new hypotheses. To promote these goals, the ImmPort data repository was created for the broader research community to explore the wide spectrum of clinical and basic research data and associated findings. The ImmPort ecosystem consists of four components-Private Data, Shared Data, Data Analysis, and Resources-for data archiving, dissemination, analyses, and reuse. To date, more than 300 studies have been made freely available through the Shared Data portal (www.immport.org/immport-open), which al

Loops through all the different IDs and the scraping material that was gathered using the above logic

In [8]:
all_articles = []
for articleID in article_ids:
   
    get_url = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id="
    get_url = get_url + articleID
    
    _create_unverified_https_context = ssl._create_unverified_context
    response = requests.get(get_url)

    soup_object = BeautifulSoup(response.content, "xml")
    one_article = get_elements(soup_object)
    all_articles.append(one_article)

Sanchita Bhattacharya, Patrick Dunn, Cristel G Thomas, Barry Smith, Henry Schaefer, Jieming Chen, Zicheng Hu, Kelly A Zalocusky, Ravi D Shankar, Shai S Shen-Orr, Elizabeth Thomson, Jeffrey Wiser, Atul J Butte

Immunology researchers are beginning to explore the possibilities of reproducibility, reuse and secondary analyses of immunology data. Open-access datasets are being applied in the validation of the methods used in the original studies, leveraging studies for meta-analysis, or generating new hypotheses. To promote these goals, the ImmPort data repository was created for the broader research community to explore the wide spectrum of clinical and basic research data and associated findings. The ImmPort ecosystem consists of four components-Private Data, Shared Data, Data Analysis, and Resources-for data archiving, dissemination, analyses, and reuse. To date, more than 300 studies have been made freely available through the Shared Data portal (www.immport.org/immport-open), which al

Ning Xu, Zhi-Bin Ke, Xiao-Dan Lin, Ye-Hui Chen, Yu-Peng Wu, Yu Chen, Ru-Nan Dong, Shao-Hao Chen, Xiao-Dong Li, Yong Wei, Qing-Shui Zheng, Yun-Zhi Lin, Xue-Yi Xue

Bladder cancer (BCa) is one of the important tumors that have been proven to be treatable with immunotherapy. This study aims to identify and validate a molecular prognostic index of BCa based on immunogenomic landscape analysis.

Sihan Chen, G D Cao, Wu Wei, Lu Yida, He Xiaobo, Yang Lei, Chen Ke, Bo Chen, Mao Ming Xiong

Colon adenocarcinoma (COAD) is a gastrointestinal tumor with a high degree of malignancy. Its deterioration process is closely related to the tumor microenvironment, and transcription factors (TF) play a regulatory role in this process. Currently, there is a lack of exploration between the genes related to the COAD tumor microenvironment and the survival prognosis of patients. Models composed of multiple genes usually predict the survival prognosis of patients more accurately than single genes. We can analyz

Qian Song, Jun Shang, Zuyi Yang, Lanlin Zhang, Chufan Zhang, Jianing Chen, Xianghua Wu

Lung cancer has become the most common cancer type and caused the most cancer deaths. Lung adenocarcinoma (LUAD) is one of the major type of lung cancer. This study aimed to establish a signature based on immune related genes that can predict patients' OS for LUAD.

Zhongyu Wang, Qian Song, Zuyi Yang, Jianing Chen, Jun Shang, Wen Ju

The kidney renal papillary cell carcinoma (KIRP) is a relatively rare type of kidney cancer. There has been no investigation to find a robust signature to predict the survival outcome of KIRP patients in the aspect of tumor immunology. In this study, 285 KIRP samples from The Cancer Genome Atlas (TCGA) were randomly divided into training and testing set. A total of 1534 immune-related genes from The Immunology Database and Analysis Portal (ImmPort) were used as candidates to construct the signature. Using univariate Cox analysis, we evaluated the relationship between ov

Randi Vita, James A Overton, Patrick Dunn, Kei-Hoi Cheung, Steven H Kleinstein, Alessandro Sette, Bjoern Peters

An Immune Exposure is the process by which components of the immune system first encounter a potential trigger. The ability to describe consistently the details of the Immune Exposure process was needed for data resources responsible for housing scientific data related to the immune response. This need was met through the development of a structured model for Immune Exposures. This model was created during curation of the immunology literature, resulting in a robust model capable of meeting the requirements of such data. We present this model with the hope that overlapping projects will adopt and or contribute to this work.
<ELocationID EIdType="pii" ValidYN="Y">baaa016</ELocationID>
DOI 10.1093/database/baaa016.
Harry Pickering, Andy Teng, Nkoyo Faal, Hassan Joof, Pateh Makalo, Eunice Cassama, Meno Nabicassa, Anna R Last, Sarah E Burr, Sarah L Rowland-Jones, Nicholas R Thom

Douglas M Molina, Sukumar Pal, Mathew A Kayala, Andy Teng, Paul J Kim, Pierre Baldi, Philip L Felgner, Xiaowu Liang, Luis M de la Maza

Chlamydia trachomatis is the most common bacterial sexually transmitted pathogen in the world. In order to control this infection there is an urgent need to formulate a vaccine. Identification of protective antigens is required to implement a subunit vaccine. To identify potential antigen vaccine candidates, three strains of mice, BALB/c, C3H/HeN and C57BL/6, were inoculated with live and inactivated C. trachomatis mouse pneumonitis (MoPn) by different routes of immunization. Using a protein microarray, serum samples collected after immunization were tested for the presence of antibodies against specific chlamydial antigens. A total of 225 open reading frames (ORF) of the C. trachomatis genome were cloned, expressed, and printed in the microarray. Using this protein microarray, a total of seven C. trachomatis dominant antigens were identified (TC0052, 

Converts the gathered data and exports it as a CSV file

In [40]:
data_frame = pd.DataFrame(all_articles)
data_frame.columns = ["Authors", "ArticleTitle", "JournalTitle", "JournalVolume", "JournalDate", "PUBID", "Abstact", "DOI"]

csv_name = keyword + ".csv"
data_frame.to_csv(csv_name)

Find all the articles that have cited a specific one.

In [19]:
def retrieve_citations(id_str):
    api_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id="  
    api_url = api_url + id_str
    print(api_url)
    
    _create_unverified_https_context = ssl._create_unverified_context
    response = requests.get(api_url)
    
    soup_object = BeautifulSoup(response.content, "xml")
    
    linked_id_list = []
    
    link_list = soup_object.find_all("Link")
    for link in link_list:
        if "Id" in str(link):
            text = link.text.replace("\n", "")
            linked_id_list.append(text)
    print(linked_id_list)
    return linked_id_list
    


In [20]:
retrieve_citations("24791905")

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id=24791905
['32699529', '32624696', '32588760', '32569190', '32537435', '32411773', '32411181', '32355728', '32352015', '32351547', '32322240', '32309184', '32296685', '32296631', '32291385', '32286381', '32219223', '32191631', '32179831', '32158466', '32143735', '32117435', '32081859', '32046766', '32039832', '32028264', '32019546', '32004409', '32002550', '31929113', '31892734', '31865910', '31842801', '31824866', '31821170', '31741756', '31695415', '31633216', '31416125', '31414702', '31371577', '31277152', '31143520', '30945466', '30852461', '30832680', '30661062', '30622534', '30594555', '30516029', '30398470', '30357391', '30304689', '30169745', '30153793', '30067990', '29921729', '29912209', '29769441', '29688354', '29490162', '29485622', '29346583', '29325141', '29322913', '29259604', '29228196', '29163494', '29144493', '29141660', '28968677', '28959737', '28925997', '28687838

['32699529',
 '32624696',
 '32588760',
 '32569190',
 '32537435',
 '32411773',
 '32411181',
 '32355728',
 '32352015',
 '32351547',
 '32322240',
 '32309184',
 '32296685',
 '32296631',
 '32291385',
 '32286381',
 '32219223',
 '32191631',
 '32179831',
 '32158466',
 '32143735',
 '32117435',
 '32081859',
 '32046766',
 '32039832',
 '32028264',
 '32019546',
 '32004409',
 '32002550',
 '31929113',
 '31892734',
 '31865910',
 '31842801',
 '31824866',
 '31821170',
 '31741756',
 '31695415',
 '31633216',
 '31416125',
 '31414702',
 '31371577',
 '31277152',
 '31143520',
 '30945466',
 '30852461',
 '30832680',
 '30661062',
 '30622534',
 '30594555',
 '30516029',
 '30398470',
 '30357391',
 '30304689',
 '30169745',
 '30153793',
 '30067990',
 '29921729',
 '29912209',
 '29769441',
 '29688354',
 '29490162',
 '29485622',
 '29346583',
 '29325141',
 '29322913',
 '29259604',
 '29228196',
 '29163494',
 '29144493',
 '29141660',
 '28968677',
 '28959737',
 '28925997',
 '28687838',
 '28652380',
 '28637268',
 '28485405',

In [21]:
len(retrieve_citations("24791905"))

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id=24791905
['32699529', '32624696', '32588760', '32569190', '32537435', '32411773', '32411181', '32355728', '32352015', '32351547', '32322240', '32309184', '32296685', '32296631', '32291385', '32286381', '32219223', '32191631', '32179831', '32158466', '32143735', '32117435', '32081859', '32046766', '32039832', '32028264', '32019546', '32004409', '32002550', '31929113', '31892734', '31865910', '31842801', '31824866', '31821170', '31741756', '31695415', '31633216', '31416125', '31414702', '31371577', '31277152', '31143520', '30945466', '30852461', '30832680', '30661062', '30622534', '30594555', '30516029', '30398470', '30357391', '30304689', '30169745', '30153793', '30067990', '29921729', '29912209', '29769441', '29688354', '29490162', '29485622', '29346583', '29325141', '29322913', '29259604', '29228196', '29163494', '29144493', '29141660', '28968677', '28959737', '28925997', '28687838

110

This creates a dictionary linking each article with the articles that cited the main article.

Later will be used along side the Zotero application

In [14]:
main_dict = {}
for id_str in article_ids:
    citation_list = retrieve_citations(id_str)
    print(id_str)
    main_dict[str(id_str)] = citation_list
    ##retrieve_citations("24791905")
    
print(main_dict)

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id=29485622&id=29485622
['32670279', '32600423', '32579175', '32547539', '32508884', '32426497', '32420368', '32283555', '32235004', '32161584', '32143735', '32081859', '32004409', '31927791', '31921619', '31787983', '31722210', '31661138', '31636302', '31443155', '31347163', '31272390', '31272374', '31231652', '31201239', '31200807', '30977847', '30852461', '30796226', '30721153', '30464745', '30398470', '30169745', '30061595', '32670279', '32600423', '32579175', '32547539', '32508884', '32426497', '32420368', '32283555', '32235004', '32161584', '32143735', '32081859', '32004409', '31927791', '31921619', '31787983', '31722210', '31661138', '31636302', '31443155', '31347163', '31272390', '31272374', '31231652', '31201239', '31200807', '30977847', '30852461', '30796226', '30721153', '30464745', '30398470', '30169745', '30061595']
29485622
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/el

['31829938', '31829938']
30398470
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id=27377652&id=27377652
['32651388', '32587977', '32449934', '32140729', '31796553', '31701156', '31651362', '31598718', '31548717', '31501545', '31272389', '31272374', '31272372', '31249003', '31028376', '30832570', '30407550', '30357391', '30252093', '29590361', '29348912', '29322915', '29322914', '29322913', '29322912', '29177046', '28682240', '28469412', '28003147', '27899649', '27899636', '32651388', '32587977', '32449934', '32140729', '31796553', '31701156', '31651362', '31598718', '31548717', '31501545', '31272389', '31272374', '31272372', '31249003', '31028376', '30832570', '30407550', '30357391', '30252093', '29590361', '29348912', '29322915', '29322914', '29322913', '29322912', '29177046', '28682240', '28469412', '28003147', '27899649', '27899636']
27377652
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed

A test to see if Zotero can be automized.

The code below makes a txt file, which is the format required to make zotero automated.

In [15]:
with open('references.txt', 'w') as json_file:
  json.dump(main_dict, json_file)

Right now the id used is self-chosen but can be implemented with automation later.

Given a list of ids, the code makes a text file with the URLs of the IDs.

In [22]:
def makeURLS(article_ids):
    global_string = ""
    count = 0
    for id_str in article_ids:
        if not count == (len(article_ids) - 1):
            global_string = global_string + "https://pubmed.ncbi.nlm.nih.gov/" + id_str + "/" + "\n"
        else:
           global_string = global_string + "https://pubmed.ncbi.nlm.nih.gov/" + id_str + "/"
        count = count + 1
    return global_string

A txt file is made and code is called.
- With the implementation the name of the file will also be changed depended on the input provided

In [23]:
reference_urls = open("test_id.txt","w+")
reference_urls.write(makeURLS(retrieve_citations("24791905")))
reference_urls.close()

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id=24791905
['32699529', '32624696', '32588760', '32569190', '32537435', '32411773', '32411181', '32355728', '32352015', '32351547', '32322240', '32309184', '32296685', '32296631', '32291385', '32286381', '32219223', '32191631', '32179831', '32158466', '32143735', '32117435', '32081859', '32046766', '32039832', '32028264', '32019546', '32004409', '32002550', '31929113', '31892734', '31865910', '31842801', '31824866', '31821170', '31741756', '31695415', '31633216', '31416125', '31414702', '31371577', '31277152', '31143520', '30945466', '30852461', '30832680', '30661062', '30622534', '30594555', '30516029', '30398470', '30357391', '30304689', '30169745', '30153793', '30067990', '29921729', '29912209', '29769441', '29688354', '29490162', '29485622', '29346583', '29325141', '29322913', '29259604', '29228196', '29163494', '29144493', '29141660', '28968677', '28959737', '28925997', '28687838